In [81]:
award_names = ['cecil b. demille award',
 'best motion picture - drama',
 'best performance by an actress in a motion picture - drama',
 'best performance by an actor in a motion picture - drama',
 'best motion picture - comedy or musical',
 'best performance by an actress in a motion picture - comedy or musical',
 'best performance by an actor in a motion picture - comedy or musical',
 'best animated feature film',
 'best foreign language film',
 'best performance by an actress in a supporting role in a motion picture',
 'best performance by an actor in a supporting role in a motion picture',
 'best director - motion picture',
 'best screenplay - motion picture',
 'best original score - motion picture',
 'best original song - motion picture',
 'best television series - drama', 'best performance by an actress in a television series - drama',
 'best performance by an actor in a television series - drama',
 'best television series - comedy or musical',
 'best performance by an actress in a television series - comedy or musical',
 'best performance by an actor in a television series - comedy or musical',
 'best mini-series or motion picture made for television',
 'best performance by an actress in a mini-series or motion picture made for television',
 'best performance by an actor in a mini-series or motion picture made for television',
 'best performance by an actress in a supporting role in a series, mini-series or motion picture made for television',
 'best performance by an actor in a supporting role in a series, mini-series or motion picture made for television']


In [82]:
import numpy as np
import pandas as pd
import spacy
from spacy import displacy
from collections import defaultdict

In [83]:
df = pd.read_json('../gg2015.json')
df = df['text']
df = df[df.str.contains('present|announc|introduc', case = False)]
df = df[~df.str.contains('think|should|maybe|RT @', case = False)]
# df = df[df.str.contains('best', case = False)]
df = df.str.replace('http\S+|www.\S+', '', case=False,regex=True)
df = df.str.replace('TV|tv', "television", case = False, regex = True)
wins_df = df.str.replace('#GoldenGlobes|#GoldenGlobe|golden|globes|globes', "", case = False, regex = True)
if wins_df.size > 8000:
    wins_df = wins_df.sample(8000)
print(len(wins_df))

8000


In [84]:
from spacy.matcher import Matcher
from spacy.tokens import Span
nlp = spacy.load('en_core_web_md')
from string import punctuation

In [85]:
# pattern1 = [{"ENT_TYPE": "PERSON", 'OP':'+'}, {"LEMMA": "win"}, {"ORTH":'Best'}, {"DEP": 'compound', 'OP':'+'}, {"POS":'NOUN', 'OP':'*'}, {'IS_PUNCT': True, 'OP':'?'}, {"POS": 'PROPN', 'OP':'*'}]
# pattern2 = [{"ENT_TYPE": "WORK_OF_ART", 'OP':'+'}, {"LEMMA": "win"}, {"ORTH":'Best'}, {"DEP": 'compound', 'OP':'+'}, {"POS":'NOUN', 'OP':'*'}, {'IS_PUNCT': True, 'OP':'?'}, {"POS": 'PROPN', 'OP':'*'}]
# pattern3 = [{"ENT_TYPE": "PERSON", 'OP':'+'}, {"LEMMA": "win"}, {"ORTH":'Best'}, {"ENT_TYPE": "WORK_OF_ART", 'OP':'+'}]
# pattern4 = [{"ENT_TYPE": "WORK_OF_ART", 'OP':'+'}, {"LEMMA": "win"}, {"ORTH":'Best'}, {"ENT_TYPE": "WORK_OF_ART", 'OP':'+'}]
# pattern5 = [{"ORTH":'Best'}, {"ENT_TYPE": "WORK_OF_ART", 'OP':'+'},{"TEXT": "goes"}, {"TEXT": "to"}, {"ENT_TYPE": "PERSON", 'OP':'+'}] 
# pattern6 = [{"ORTH":'Best'}, {"DEP": 'compound', 'OP':'+'}, {"POS":'NOUN', 'OP':'*'}, {'IS_PUNCT': True, 'OP':'?'}, {"POS": 'PROPN', 'OP':'*'}, {"TEXT": "goes"}, {"TEXT": "to"}, {"ENT_TYPE": "PERSON", 'OP':'+'}] 

person_pattern = [{"ENT_TYPE": "PERSON", 'OP':'+'}]
film_pattern = [{"ENT_TYPE": 'ORG', 'OP':'+'}]

award_pattern = [{"LOWER":'best'}, {"DEP":{"IN":['compound','nmod','dobj','prep','det','pobj','amod','nsubj','ROOT']}, 'OP':'+'},{'ORTH': '-', 'OP':'?'}, {"POS": {"IN": ['PROPN','NOUN','CCONJ','ADJ']}, 'OP':'*'}]

In [86]:
doc = nlp('Skyfall wins best motion picture drama - Skyfall')
for entity in doc.ents:
    print(entity.text, entity.label_)

Skyfall ORG


In [87]:
matcher = Matcher(nlp.vocab)
matcher.add('Person',[person_pattern])
matcher.add('Award',[award_pattern])
matcher.add('Film',[film_pattern])
# matches = matcher(doc)

In [88]:
def find_mapping(matches):
    winner_len = 0
    award_len = 0
    winner = ''
    award = ''
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]
        span = doc[start:end]
        span_len = len(span.text)
        if string_id == 'Person' and len(span)<= 2:
            if winner_len < span_len:
                winner = span.text
                winner_len = span_len      
        elif string_id == 'Award':
            if award_len < span_len:
                award = span.text
                award_len = span_len
    winner = str(winner).strip(punctuation).strip()
    award = str(award).strip(punctuation).strip()
    key = (winner.lower(),award)
    return key

In [89]:
winners = {}
winners = defaultdict(lambda: 0, winners)
for i, text in wins_df.iteritems():
    doc = nlp(text)
    matches = matcher(doc)
    if len(matches) != 0:
        key = find_mapping(matches)
        if (key[0] != '' and key[1] != ''):
            winners[key] = winners[key] +1

In [90]:
# winners = sorted(winners.items(), key=lambda item: item[1], reverse = True)
winners = dict(winners)
print(winners)

{('ricky', 'Best Actress in a Motion Picture'): 1, ('prince presents', 'Best Original Song for'): 55, ('jeremy renner', 'Best presenters'): 1, ('common', 'Best Original Song'): 1, ('lily tomlin', 'Best Actor in a comedy'): 1, ('jeffrey tambor', 'Best Actor in a television Series'): 4, ('common', 'Best of Hip-Hop'): 1, ('jamie dorman', 'best pair of presenters of history in the'): 1, ('lil sebastian', 'Best Mini Horse'): 1, ('john legend', 'Best Original Song'): 7, ('su', 'Best Original Score - Motion Picture'): 2, ('adrien brody', 'Best Actress in a Miniseries or television Movie'): 1, ('kevin hart', 'Best Animated movie'): 1, ('prince', 'best song in a motion picture'): 1, ('gina rodriguez', 'Best Actress in a television series comedy goes to'): 2, ('jared leto', 'best Supporting Actress in Film'): 1, ('selma hayek', 'Best Animated Feature Film to'): 1, ('bryan cranston', 'best television drama and Bryan Cranston'): 1, ('lily tomlin', 'Best Actor in Comedy or Musical'): 1, ('beyonce',

In [91]:
sort_d = sorted(winners.items(), key=lambda item: item[1], reverse = True)
print(sort_d)

[(('prince presents', 'Best Original Song for'), 55), (('john legend', 'Best Original Song'), 7), (('benedict', 'best actor at the'), 7), (('john legend', 'Best Original Song - Motion Picture'), 6), (('birdman wins', 'Best Screenplay'), 6), (('prince', 'Best Original Song'), 6), (('benedict', 'best actor'), 6), (('julianne moore', 'Best Actress in a Motion Picture'), 6), (('eddie redmayne', 'Best Actor in a Motion Picture'), 5), (('robert downey', 'Best Movie'), 5), (('jeffrey tambor', 'Best Actor in a television Series'), 4), (('michael keaton', 'Best Actor in a Motion Picture'), 4), (('richardlinklater', 'Best Picture'), 4), (('patricia arquette', 'Best Supporting Actress in a Motion Picture'), 4), (('kristen wiig', 'Best Screenplay'), 4), (('meryl streep', 'Best Motion Picture'), 4), (('robert downey', 'Best Film - Comedy'), 3), (('oprah winfrey', 'best picture nominee'), 3), (('sund', 'Best Foreign Language Film'), 3), (('adams wins', 'Best Actress in a Motion Picture'), 3), (('jen

In [92]:
import Levenshtein
def map_to_a_award(tweet):
    awards = {}
    tweet = tweet.lower()
    for award in award_names:
        split_award = award.split()
        award_dist = 0
        split_tweet = tweet.split()
        for word in tweet.split():
            min_val = float('inf')
            for word2 in award.split():
                min_val = min(min_val,Levenshtein.distance(word, word2))
            award_dist += min_val
        awards[award] = award_dist
        
    val = min(awards.values())
    res = [key for key in awards if awards[key] == val]
    return res

In [93]:
final_ans = {}
for award in award_names:
    sub_dict = {}
    sub_dict = defaultdict(lambda: 0, sub_dict)
    final_ans[award] = sub_dict
for pair,count in winners.items():
    award = pair[1]
    person = pair[0]
    possible_awards = map_to_a_award(award)
    for ard in possible_awards:
        final_ans[ard][person] = final_ans[ard][person] + count
for ans in final_ans:
    if len(final_ans[ans].items()) > 0:
        final_ans[ans] = max(final_ans[ans], key=final_ans[ans].get)
    else:
        final_ans[ans] = ''
print(final_ans)

{'cecil b. demille award': '', 'best motion picture - drama': 'meryl streep', 'best performance by an actress in a motion picture - drama': 'meryl streep', 'best performance by an actor in a motion picture - drama': 'benedict', 'best motion picture - comedy or musical': 'robert downey', 'best performance by an actress in a motion picture - comedy or musical': 'robert downey', 'best performance by an actor in a motion picture - comedy or musical': 'robert downey', 'best animated feature film': 'selma hayek', 'best foreign language film': 'sund', 'best performance by an actress in a supporting role in a motion picture': 'robert downey', 'best performance by an actor in a supporting role in a motion picture': 'benedict', 'best director - motion picture': 'robert downey', 'best screenplay - motion picture': 'kristen wiig', 'best original score - motion picture': 'robert downey', 'best original song - motion picture': 'prince presents', 'best television series - drama': 'adam levine', 'best